# Лекция №3 (часть 1)

На лекции я показывал работу с отладчиком в VS Code.

В формате Jupyter Notebook я не могу показать то же самое, поэтому примеры здесь отличаются.

## Пространства имён (namespace)

### The Zen of Python

In [1]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


Обратите внимание на последнюю строку:
“Namespaces are one honking great idea — let's do more of those!”

Что это значит?

__Пространство имён — это словарь__, ключи которого — это имена, а значения — объекты (числа, строки, списки, словари, множества, функции...).

### Пространства выполнения кода

Отдельные пространства имён создаются при выполнении кода внутри:
* модуля (файла),
* функции,
* безымянной функции (`lambda x: x + 1`),
* безымянных выражений типа `[x + 1 for x in iterator]` (“list|set|dict comprehensions”).

In [2]:
def example_f():
    "Пример пространства имён при выполнении функции"
    a = 1
    b = 2
    print(locals())  # распечатать локальное пространство имён
    return a + b

example_f();

{'a': 1, 'b': 2}


Пространство имён внутри модуля слишком велико, рассмотрим отдельные его элементы:

In [3]:
# Некоторые элементы пространства имён исполняемого модуля
print( locals()["__name__"] )  # имя модуля
print( locals()["__doc__"] )   # строка документации к модулю
print( locals()["this"] )      # импортированный выше модуль
print( locals()["example_f"] ) # определённая выше функция

__main__
Automatically created module for IPython interactive environment
<module 'this' from '/usr/local/Cellar/python@3.11/3.11.5/Frameworks/Python.framework/Versions/3.11/lib/python3.11/this.py'>
<function example_f at 0x10b3a1080>


List comprehensions:

In [4]:
def example_g():
    "Пример пространства имён при выполнении 'list comprehension'"
    result = [
        print(f"Inside: { locals() }") # распечатать локальное пространство имён
        for x in range(2)
        for y in range(2)
    ]
    print(f"Outside: { locals() }") # распечатать локальное пространство имён
    return result

example_g();

Inside: {'.0': <range_iterator object at 0x10b386610>, 'x': 0, 'y': 0}
Inside: {'.0': <range_iterator object at 0x10b386610>, 'x': 0, 'y': 1}
Inside: {'.0': <range_iterator object at 0x10b386610>, 'x': 1, 'y': 0}
Inside: {'.0': <range_iterator object at 0x10b386610>, 'x': 1, 'y': 1}
Outside: {'result': [None, None, None, None]}


### Пространства имён при объектах

Кроме этого, своё пространство имён есть:
* у каждого объекта,
* у каждого класса объектов.

__Внимание: специальный синтаксис!__ Вместо `obj["name"]` пишется `obj.name`.

Открытые файлы — это объекты:

In [5]:
with open("example.txt", "w", encoding="utf-8") as file:
    # объект-файл
    print( file )

<_io.TextIOWrapper name='example.txt' mode='w' encoding='utf-8'>


In [6]:
with open("example.txt", "w", encoding="utf-8") as file:
    # Некоторые элементы пространства имён у объекта-файла
    print( file.__class__ ) # класс объекта-файла
    print( file.name )      # имя файла
    print( file.mode )      # чтение или запись?
    print( file.encoding )  # кодировка

<class '_io.TextIOWrapper'>
example.txt
w
utf-8


Функции — это тоже объекты.

In [7]:
# объект-функция
print( example_f )

<function example_f at 0x10b3a1080>


In [8]:
# Некоторые элементы пространства имён у объекта-функции
print( example_f.__name__ )   # имя функции при определении
print( example_f.__doc__ )    # строка документации
print( example_f.__module__ ) # модуль, где была определена функция
print( example_f.__class__ )  # класс объекта-функции

example_f
Пример пространства имён при выполнении функции
__main__
<class 'function'>


Мы будем проходить классы и объекты позже. Но, забегая вперёд, ещё несколько примеров:

In [9]:
class MyClass:
    "Пример класса."

    # пример поля
    num = 42

In [10]:
# наш класс
print( MyClass )

<class '__main__.MyClass'>


In [11]:
# Некоторые элементы пространства имён у нашего класса
print( MyClass.__name__ )     # имя класса при определении
print( MyClass.__doc__ )      # строка документации
print( MyClass.__class__ )    # класс объекта-класса
print( MyClass.num )          # наш пример

MyClass
Пример класса.
<class 'type'>
42


In [12]:
my_object = MyClass()

# пусть имя "s" указывает на строку "abc"
my_object.s = "abc"

In [13]:
# объект
print( my_object )

In [14]:
# Некоторые элементы пространства имён у нашего объекта
print( my_object.__class__ )  # класс объекта
print( my_object.s )          # наш пример

<class '__main__.MyClass'>
abc


## Матрёшки

### Вложенные пространства выполнения кода

__Общее правило:__ когда поток выполнения находится в одном пространстве имён, он не "видит" имена из других пространств имён.

__Исключение:__ поток выполнения "видит" имена из того пространства, внутри которого было определено (создано) текущее пространство.
* изнутри функции видно пространство имён модуля, где она была определена,
* изнутри вложенной функции видно пространство имён родительской функции.

In [15]:
def outer_function():
    "Определение внешней функции."

    # определим имя 'X' во внешнем пространстве имён
    X = 1

    def inner_function():
        "Определение внутренней функции."
        # определим имя 'Y' во внутреннем пространстве имён
        Y = 2

        # что мы видим изнутри?
        print( f"Изнутри всё видно: X = {X}, Y = {Y}." )

    # внешняя функция запускает внутреннюю
    inner_function()

In [16]:
outer_function()

Изнутри всё видно: X = 1, Y = 2.


__Изнутри нельзя нечаянно изменить привязку внешнего имени к объекту.__ Можно создать новое внутреннее имя, которое закрывает (затеняет) собой внешнее.

In [17]:
def another_outer_function():
    "Определение внешней функции."

    # определим имя 'X' во внешнем пространстве имён
    X = 1

    def inner_function():
        "Определение внутренней функции."
        X = 2
        print( f"Изнутри: X = {X}." )

    # внешняя функция запускает внутреннюю
    inner_function()

    print( f"Снаружи: X = {X}." )

In [18]:
another_outer_function()

Изнутри: X = 2.
Снаружи: X = 1.


Во избежание путаницы Питон запрещает использовать одно и то же имя сначала как внешнее, затем как внутреннее. Следующий пример не работает:

In [19]:
def yet_another_outer_function():
    "Определение внешней функции."
    X = 1

    def inner_function():
        "Определение внутренней функции."
        print( f"Используем имя 'X' как внешнее: {X}." )
        X = 2
        print( f"Используем имя 'X' как внутреннее: {X}." )

    # внешняя функция запускает внутреннюю
    inner_function()

In [20]:
yet_another_outer_function()

UnboundLocalError: cannot access local variable 'X' where it is not associated with a value

Если мы сознательно желаете изменить значение внешнего имени, это делается ключевым словом `nonlocal` (для имён из модуля — `global`).

In [21]:
def outer_function_with_mutation():
    "Определение внешней функции."
    X = 1

    def inner_function():
        "Определение внутренней функции."
        nonlocal X # да, я осознанно меняю значение внешнего имени
        X = 2

    # внешняя функция запускает внутреннюю
    inner_function()
    print( f"Снаружи: X = {X}." )

In [22]:
outer_function_with_mutation()

Снаружи: X = 2.


### Продвинутый пример

In [23]:
def create_inner_function():
    "Контекст, где внутренняя функция определена."

    X = 1

    def inner_function():
        "Определение внутренней функции."
        print( f"X = {X}" )

    return inner_function
    

In [24]:
def use_inner_function(func):
    "Контекст, где внутренняя функция используется."
    X = 2
    func()

In [25]:
# запускаем функцию 'create_inner_function'
# получаем оттуда функцию 'inner_function' (как объект)
func = create_inner_function()

# передаём 'inner_function' внутрь 'use_inner_function'
# запускаем её там внутри
use_inner_function(func)

X = 1
